In [1]:
from pathlib import Path
import importlib.util
import threading
import pyautogui
import time
# parent directory of student repos
d = '/Users/charlesliu/Documents/HarvardTeaching/CS207'

In [2]:
# aggregate student repos in directory
p = Path(d)
studentrepos = []
for x in p.iterdir():
    repo = str(x).replace(d + '/', '').lower()
    if 'cs207' in repo:
        studentrepos.append((x, repo))
studentrepos

[(PosixPath('/Users/charlesliu/Documents/HarvardTeaching/CS207/cs207_david_sondak'),
  'cs207_david_sondak')]

In [17]:
# class for a thread to enter user input for the atm session
class TypewriteThread(threading.Thread):
    def __init__(self, msg, interval=0.0, sleep=0.25):
        super(TypewriteThread, self).__init__()
        self.msg = msg
        self.interval = interval
        self.sleep = sleep


    def run(self):
        time.sleep(self.sleep)
        pyautogui.typewrite(self.msg, self.interval)

In [34]:
# tests to run given module, return score
def testSuite (m):
    SAVINGS = m.AccountType.SAVINGS
    CHECKING = m.AccountType.CHECKING
    # create a bank account and bank user, just check no errors thrown
    try:
        acc = m.BankAccount('Charles Liu', m.AccountType.SAVINGS)
        user = m.BankUser('Charles Liu')
    except:
        # if this doesn't work then it's a 0
        return '0 (Classes didnt work)'
    
    # run some basic bank account functions
    try:
        if len(acc) != 0:
            raise ValueError('balance not starting at 0')
        try:
            acc.withdraw(100)
        except:
            pass
        if len(acc) != 0:
            raise ValueError('withdraw did something when it shouldnt have')
        acc.deposit(100)
        if len(acc) != 100:
            raise ValueError('deposit didnt work correctly')
        acc.withdraw(50)
        if len(acc) != 50:
            raise ValueError('withdraw didnt work correctly')
    except ValueError as e:
        return '1 (' + str(e) + ')'
    
    # run some basic bank user functions
    try:
        try:
            savings = user.getBalance(SAVINGS)
            checking = user.getBalance(CHECKING)
            if savings >= 0 and (type(savings) == int or type(savings) == float):
                raise ValueError('account hasnt been created yet but returning a number on getbalance')
            if checking >= 0 and (type(checking) == int or type(checking) == float):
                raise ValueError('account hasnt been created yet but returning a number on getbalance')
        except:
            pass
        
        user.addAccount(SAVINGS)
        user.addAccount(CHECKING)
        
        if user.getBalance(SAVINGS) != 0 or user.getBalance(CHECKING) != 0:
            raise ValueError('invalid balance after adding account')
            
        user.deposit(SAVINGS, 1000)
        user.deposit(CHECKING, 500)

        if user.getBalance(SAVINGS) != 1000 or user.getBalance(CHECKING) != 500:
            raise ValueError('invalid balance after depositing')
            
        user.withdraw(SAVINGS, 500)
        if user.getBalance(SAVINGS) != 500 or user.getBalance(CHECKING) != 500:
            raise ValueError('invalid balance after withdrawing')
            
        try:
            user.addAccount(SAVINGS)
        except:
            pass
        
        if user.getBalance(SAVINGS) != 500:
            raise ValueError('adding account second time overwrote balance')
        
    except ValueError as e:
        return '3 (' + str(e) + ')'
    
    # closure is a bit harder to automate since the interface may be different if student was ambitious
    # we'll just check that an operation works for the user we created
    try:
        interface = m.ATMSession(user)
        # Choose Deposit
        TypewriteThread(['4', '\n'], sleep=0.5).start()
        # Choose checking
        TypewriteThread(['1', '\n'], sleep=1.0).start()
        # Deposit 100
        TypewriteThread(['1', '0', '0', '\n'], sleep=1.5).start()
        # Exit prompt
        TypewriteThread(['1', '\n'], sleep=2.0).start()
        interface()
        if user.getBalance(CHECKING) != 600:
            raise ValueError('ATM Session didnt deposit correctly')
    except ValueError as e:
        return '4 (' + str(e) + ')'
        
    return 5

In [35]:
# run script
grades = {}
for path, repo in studentrepos:
    hw6 = path / 'homeworks' / 'HW6' / 'Bank.py'
    spec = importlib.util.spec_from_file_location("Bank", str(hw6.resolve()))
    b = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(b)
    grades[repo] = testSuite(b)
grades

Enter Option:
1)Exit
2)Create Account
3)Check Balance
4)Deposit
5)WithDraw

4
Select Account Type:
1) Checking
2) Savings
1
Enter integer amount, cannot be negative
100
Transaction was successful

Charles Liu's Accounts:
CHECKING: 600
SAVINGS: 500

Enter Option:
1)Exit
2)Create Account
3)Check Balance
4)Deposit
5)WithDraw

1
Charles Liu's Accounts:
CHECKING: 600
SAVINGS: 500



{'cs207_david_sondak': 5}